In [ ]:
import os
import torch
import pandas as pd
from datasets import load_dataset, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    AutoConfig
)
import glob
import multiprocessing

if __name__ == '__main__':
    print("Setting up simple Llama training...")

    # Configuration
    TRAIN_DATA_PATH = './data/subtaskA_train_monolingual.jsonl'
    TEST_DATA_PATH = './data/subtaskA_monolingual.jsonl'
    MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # TinyLlama model
    OUTPUT_DIR = "./models/simple_trained_llama_tiny_1"
    TOKEN = ""

    # Simplified parameters
    BATCH_SIZE = 8  # Reduced batch size for stability
    EPOCHS = 1      # Just one epoch for simplicity
    MAX_LENGTH = 64  # Increased sequence length for better context
    TRAIN_SAMPLE_RATIO = 0.05  # Use only 5% of training data for faster training
    SAVE_STEPS = 10  # Save less frequently
    SAVE_TOTAL_LIMIT = 2  # Keep fewer checkpoints

    # Set Hugging Face token
    os.environ["HUGGING_FACE_HUB_TOKEN"] = TOKEN

    # Enable CUDA optimizations
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.backends.cuda.matmul.allow_tf32 = True
        print("CUDA optimizations enabled")
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

    # Check for data files
    for path in [TRAIN_DATA_PATH, TEST_DATA_PATH]:
        if not os.path.exists(path):
            alt_path = path.replace('./data/', '')
            if os.path.exists(alt_path):
                print(f"Found {path} at {alt_path}")
                if path == TRAIN_DATA_PATH:
                    TRAIN_DATA_PATH = alt_path
                else:
                    TEST_DATA_PATH = alt_path
            else:
                print(f"Warning: File not found at {path} or {alt_path}")

    # Create output directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Create checkpoint directory
    checkpoint_dir = os.path.join(OUTPUT_DIR, "checkpoints")
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Check for existing checkpoints
    checkpoint_pattern = os.path.join(checkpoint_dir, "checkpoint-*")
    checkpoints = glob.glob(checkpoint_pattern)
    if checkpoints:
        # Get the latest checkpoint
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print(f"Found existing checkpoint at {latest_checkpoint}")
        resume_from_checkpoint = latest_checkpoint
        print(f"Will resume training from step {latest_checkpoint.split('-')[-1]}")
    else:
        resume_from_checkpoint = None
        print("No existing checkpoints found, starting from scratch")

    # Load tokenizer
    print(f"Loading tokenizer for {MODEL_ID}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # Load model configuration
    print(f"Loading model configuration for {MODEL_ID}...")
    config = AutoConfig.from_pretrained(MODEL_ID)
    config.num_labels = 2  # Set number of labels for classification
    config.pad_token_id = tokenizer.pad_token_id

    # Load model for sequence classification
    print(f"Loading model {MODEL_ID}...")
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_ID,
        config=config,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True
    )

    # Set device and move model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model = model.to(device)

    # Preprocessing function with optimizations
    def preprocess(examples):
        return tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=MAX_LENGTH,
            return_tensors="pt"  # Return PyTorch tensors directly
        )

    # Load and prepare training data with optimizations
    print(f"Loading training data from {TRAIN_DATA_PATH}...")
    train_dataset = load_dataset("json", data_files=TRAIN_DATA_PATH)
    train_dataset = train_dataset["train"].shuffle(seed=42)

    # Use only a small portion of the training data
    original_size = len(train_dataset)
    selected_size = int(original_size * TRAIN_SAMPLE_RATIO)
    train_dataset = train_dataset.select(range(selected_size))
    print(f"Using {selected_size} examples out of {original_size} ({TRAIN_SAMPLE_RATIO*100}%)")

    # Clean the dataset and convert labels
    train_dataset = train_dataset.remove_columns(["model", "source"])
    train_dataset = train_dataset.cast_column("label", ClassLabel(names=["human", "machine"]))

    # Check class balance
    human_count = sum(1 for item in train_dataset if item['label'] == 0)
    machine_count = sum(1 for item in train_dataset if item['label'] == 1)
    print(f"Class balance in training data: {human_count} human, {machine_count} machine")

    # Tokenize datasets with optimizations
    print("Tokenizing datasets...")
    train_tokenized = train_dataset.map(
        preprocess,
        batched=True,
        batch_size=32,  # Process in larger batches
        num_proc=1,     # Disable multiprocessing to avoid Windows issues
        remove_columns=["text", "id"]
    )

    # Simple training arguments with GPU optimizations
    print("Setting up training...")
    training_args = TrainingArguments(
        output_dir=checkpoint_dir,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        logging_steps=5,
        save_steps=SAVE_STEPS,
        save_total_limit=SAVE_TOTAL_LIMIT,
        learning_rate=1e-5,
        weight_decay=0.01,
        fp16=torch.cuda.is_available(),  # Enable mixed precision training
        bf16=torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8,  # Enable bfloat16 if supported
        report_to="none",
        push_to_hub=False,
        disable_tqdm=False,
        gradient_accumulation_steps=4,  # Reduced for faster training
        warmup_steps=5,
        max_grad_norm=1.0,
        dataloader_num_workers=0,  # Disable multiprocessing for data loading
        dataloader_pin_memory=torch.cuda.is_available(),
        optim="adamw_torch",
        lr_scheduler_type="linear",
        use_cpu=not torch.cuda.is_available(),
        save_strategy="steps",
        save_safetensors=True,
        save_on_each_node=True,
        overwrite_output_dir=False,
    )

    # Create trainer with optimizations
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
    )

    # Train the model with progress tracking
    print("Starting training...")
    try:
        # Load the latest checkpoint if available
        if resume_from_checkpoint:
            print(f"Loading checkpoint from {resume_from_checkpoint}")
            trainer.train(resume_from_checkpoint=resume_from_checkpoint)
        else:
            print("Starting fresh training")
            trainer.train()

        print("Training completed successfully!")

        # Save final model
        print(f"Saving final model to {OUTPUT_DIR}")
        trainer.save_model(OUTPUT_DIR)

    except Exception as e:
        print(f"Training interrupted: {str(e)}")
        print("You can resume training later by running this script again.")
        print("The latest checkpoint will be automatically detected.")
        raise e

    # Load test data
    print(f"Loading test data from {TEST_DATA_PATH}...")
    test_dataset = load_dataset("json", data_files={"test": TEST_DATA_PATH})["test"]

    # Tokenize test dataset with optimizations
    print("Tokenizing test dataset...")
    test_tokenized = test_dataset.map(
        preprocess,
        batched=True,
        batch_size=32,  # Process in larger batches
        num_proc=1,     # Disable multiprocessing to avoid Windows issues
        remove_columns=test_dataset.column_names  # Remove unnecessary columns
    )

    # Make predictions with optimizations
    print("Making predictions...")
    predictions = trainer.predict(
        test_tokenized,
        batch_size=16,  # Larger batch size for prediction
        max_length=MAX_LENGTH,
        num_workers=0,  # Disable multiprocessing
        use_cache=True  # Cache predictions
    )

    # Process predictions efficiently
    predicted_labels = predictions.predictions.argmax(-1)

    # Format and save predictions efficiently
    print("Saving predictions...")
    predictions_df = pd.DataFrame({
        "id": test_dataset["id"],
        "label": predicted_labels
    })

    # Save predictions with optimized settings
    output_path = os.path.join(OUTPUT_DIR, "subtaskA_monolingual.jsonl")
    predictions_df.to_json(output_path, orient="records", lines=True, compression='gzip')

    print("Done! Training and prediction complete.")

In [ ]:
import json
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def load_jsonl(file_path):
    """Load a JSONL file into a list of dictionaries."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def evaluate_predictions(pred_file, gold_file):
    """Evaluate predictions against gold labels."""
    # Load predictions and gold labels
    print(f"Loading predictions from {pred_file}...")
    pred_data = load_jsonl(pred_file)
    print(f"Loading gold labels from {gold_file}...")
    gold_data = load_jsonl(gold_file)

    # Convert to DataFrames
    pred_df = pd.DataFrame(pred_data)
    gold_df = pd.DataFrame(gold_data)

    # Ensure IDs match
    pred_df = pred_df.sort_values('id')
    gold_df = gold_df.sort_values('id')

    # Verify data alignment
    if not (pred_df['id'] == gold_df['id']).all():
        print("Warning: Prediction and gold file IDs don't match perfectly!")
        # Find common IDs
        common_ids = set(pred_df['id']) & set(gold_df['id'])
        pred_df = pred_df[pred_df['id'].isin(common_ids)]
        gold_df = gold_df[gold_df['id'].isin(common_ids)]

    # Get labels
    y_pred = pred_df['label'].values
    y_true = gold_df['label'].values

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_micro = f1_score(y_true, y_pred, average='micro')

    # Print results
    print("\nEvaluation Results:")
    print("-" * 50)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Macro: {f1_macro:.4f}")
    print(f"F1 Micro: {f1_micro:.4f}")

    # # Print class-wise metrics
    # print("\nClass-wise Metrics:")
    # print("-" * 50)
    # for label in [0, 1]:
    #     label_name = "Human" if label == 0 else "Machine"
    #     class_pred = (y_pred == label)
    #     class_true = (y_true == label)
    #     class_acc = accuracy_score(class_true, class_pred)
    #     class_f1 = f1_score(class_true, class_pred)
    #     print(f"\n{label_name} Class:")
    #     print(f"Accuracy: {class_acc:.4f}")
    #     print(f"F1 Score: {class_f1:.4f}")

    # # Print confusion matrix
    # print("\nConfusion Matrix:")
    # print("-" * 50)
    # from sklearn.metrics import confusion_matrix
    # cm = confusion_matrix(y_true, y_pred)
    # print("\nPredicted →")
    # print("Actual ↓")
    # print(f"           Human  Machine")
    # print(f"Human      {cm[0,0]:<6} {cm[0,1]:<6}")
    # print(f"Machine    {cm[1,0]:<6} {cm[1,1]:<6}")

if __name__ == "__main__":
    # File paths
    pred_file = "C:/Users/akshi/Downloads/CSI 5386 NLP A2/CSI 5386 NLP A2/models/simple_trained_llama_tiny_1/subtaskA_monolingual.jsonl"
    gold_file = "C:/Users/akshi/Downloads/CSI 5386 NLP A2/CSI 5386 NLP A2/data/subtaskA_monolingual_gold.jsonl"

    # Check if files exist
    import os
    if not os.path.exists(pred_file):
        print(f"Error: Prediction file not found at {pred_file}")
    elif not os.path.exists(gold_file):
        print(f"Error: Gold file not found at {gold_file}")
    else:
        evaluate_predictions(pred_file, gold_file)